In [ ]:
import random

In [ ]:
text = input()
if text in ["Привет", "Здарова", "Хеллоу"]:
  print(random.choice(["Здрасте", "Йоу", "Приветики"]))
elif text in ["Пока", "Увидимся", "Чао"]:
  print(random.choice(["Буду ждать нашей встречи", "Ок", "Бай-бай"]))
else:
  print("Не понял")

бай
Не понял


In [ ]:
import re # Regular Expressions
import nltk # Natural Language Toolkit
# filter_text - фильтрует текст
# "Привет" => "привет"
# "привет !!! :)" => "привет"
# "  привет  " => "привет"
def filter_text(text):
    text = text.lower()  # lower - к нижнему регистру
    text = text.strip()  # strip - вырезать пробелы с начала и конца строки
    pattern = r"[^\w\s]" # Все что не слово и не пробел
    text = re.sub(pattern, "", text) # Из переменной text вырезаем "Все что не слово и не пробел"
    return text

# True, если тексты похожи (user_text пользовательский текст, example - контрольная фраза)
# False, если не похожи
def text_match(user_text, example):
    user_text = filter_text(user_text) # Отфильтруем лишнее из первой строки
    example = filter_text(example) # Отфильтруем лишнее из  строки
    if user_text == example: # Если тексты точно совпадают
        return True
    if user_text.find(example) != -1: # Если фраза входит в пользовательский текст
        return True

    distance = nltk.edit_distance(user_text, example)
    # Отношение кол-ва ошибок к длине слова, 1.0 - слово целиком другое, 0 - слова полностью совпадают
    ratio = distance / len(example)  
    if ratio < 0.40:
        return True
    
    return False

In [ ]:
# Тексты совпадают
text_match("Привет", "Привет")  

True

In [ ]:
# Разный регистр (ToDo: lower)
text_match("привет", "Привет")

True

In [ ]:
# Лишние символы (ToDo: regular expressions)
text_match("Привет!!!", "Привет")  

True

In [ ]:
# Пробелы
text_match("   Привет!!!   ", "Привет")  

True

In [ ]:
# Лишние слова (ToDo: find)
text_match("чувачок Привет, какие твои делища", "Привет")  

True

In [ ]:
# Опечатки (ToDo: levenstein)
text_match("Превет", "Привет")
# Расстояние редактирования (расстояние левенштейна)

True

# Определение "намерения" пользователя (англ. intent, интент)

In [ ]:
# Составим карты "намерений", которые поддерживает наш бот
INTENTS = {
    "hello": {
        "examples": ["Привет", "Здравствуйте", "Добрый день"],
        "responses": ["Йоу", "Здарова", "Приветствую тебя, человек."],
    },
    "bye": {
        "examples": ["Пока", "До свидания", "Всего хорошего"],
        "responses": ["Давайдосвиданья", "И вам приятного денечка"],
    },
    "how_are_you": {
        "examples": ["Как дела"],
        "responses": ["Функционирую нормально"],
    },
    "work": {
        "examples": ["Работать", "Офис", "Хочу"],
        "responses": ["Скоро все получится"],
    },
}

In [ ]:
import random
# Функция которая находит намерение пользователя по его тексту (с помощью text_match)
def get_intent(user_text):
    for intent in INTENTS:
        examples = INTENTS[intent]["examples"]
        for example in examples:
            if len(filter_text(example)) < 3: 
                continue
            if text_match(user_text, example):
                return intent # Найденное намерение подходит к польз. тексту
    return None # Ничего не найдено

def get_random_response(intent):
    return random.choice(INTENTS[intent]["responses"])

In [ ]:
get_intent("Я очень хочу работать")

'work'

In [ ]:
get_random_response("bye")

'И вам приятного денечка'

In [ ]:
# Пример использования
user_text = ""
while user_text != "Пока":
    print("[USER]: ", end='')
    user_text = input()
    intent = get_intent(user_text)
    response = get_random_response(intent)
    print(f"[BOT]: {response}")

[USER]: Привет
[BOT]: Здарова
[USER]: Я очень хочу работать
[BOT]: Скоро все получится
[USER]: Пока
[BOT]: Давайдосвиданья


# Классификация текстов ML-моделью

In [ ]:
import json
with open("/content/big_bot_config.json", "r") as config_file:
    data = json.load(config_file)

INTENTS = data["intents"]

In [ ]:
print("Интентов загружено из файла:", len(INTENTS))

Интентов загружено из файла: 439


In [ ]:
# Фраза (X) - на вход
# Интент (y) - на выход
X = []
y = []

for intent in INTENTS:
    examples = INTENTS[intent]["examples"]
    for example in examples:
        example = filter_text(example)
        if len(example) < 3:
            continue # Пропускаем текст если он слишком короткий
        X.append(example) 
        y.append(intent)

In [ ]:
len(X), len(y)

(2979, 2979)

# Векторизация
 


Например: [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), [TfIDFVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() 

vectorizer.fit(X) # Обучаем векторайзер

vecX = vectorizer.transform(X) # Все тексты преобразуем в вектора

In [ ]:
X[1500], y[1500]

('где вы находитесь', 'locations')

In [ ]:
print(vecX[1500])

  (0, 414)	1
  (0, 443)	1
  (0, 1277)	1


In [ ]:
# Пример
print(vectorizer.transform(["где вы находитесь находитесь"]))

  (0, 414)	1
  (0, 443)	1
  (0, 1277)	2


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 150, max_depth = 4 )
model.fit(vecX, y) # Обучение модели

RandomForestClassifier(max_depth=4, n_estimators=150)

In [ ]:
model.predict(vectorizer.transform(["Насколько ты разумна?"]))

array(['cities'], dtype='<U29')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
y_pred = model.predict(vecX)

print("accuracy_score", accuracy_score(y, y_pred)) # Сравниваем y и y_pred
print("f1_score", f1_score(y, y_pred, average="macro")) # Сравниваем y и y_pred

accuracy_score 0.12084592145015106
f1_score 0.00147180269257807


In [ ]:
def get_intent_ml(user_text):
    user_text = filter_text(user_text)
    vec_text = vectorizer.transform([user_text])
    intent = model.predict(vec_text)[0]
    # model.predict_proba()
    return intent

In [ ]:
import pandas as pd
proba = model.predict_proba(vectorizer.transform(["Привет, как дела"]))

# Вывести на экран список вероятных интентов
pd.DataFrame(columns=model.classes_, data=[proba[0]]).T.sort_values(by=0, ascending=False)

,0
hello,0.101303
cities,0.097836
bye,0.039660
name,0.037851
byflood,0.031826
...,...
song2,0.000261
whatisintent,0.000255
song3,0.000252
pasta or rise,0.000246


In [ ]:
def bot(user_text):
    intent = get_intent(user_text)
    if intent:
        return get_random_response(intent)
    intent = get_intent_ml(user_text)
    return get_random_response(intent)


Бот в Телеграм

In [ ]:
! pip install python-telegram-bot --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 10.1 MB/s eta 0:00:00


In [ ]:
! pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nest_asyncio
nest_asyncio.apply() 

In [ ]:
TOKEN="мой токен, но я вам его не скажу"

In [ ]:
from telegram import Update 
from telegram.ext import ApplicationBuilder, MessageHandler, filters

# Создаем и настраиваем бот-приложение
app = ApplicationBuilder().token(TOKEN).build()

async def telegram_reply(upd: Update, ctx):
    name = upd.message.from_user.full_name
    user_text = upd.message.text
    print(f"{name}: {user_text}")
    reply = bot(user_text)
    print(f"BOT: {reply}")
    await upd.message.reply_text(reply)
    
handler = MessageHandler(filters.TEXT, telegram_reply) # Создаем обработчик текстовых сообщений
app.add_handler(handler) # Добавляем обработчик в приложение

# app.run_polling # Наш код регулярно опрашивает сервер на предмет новых Апдейтов
# app.run_webhook # Запускает веб-сервер, к которому будет подключаться сам ТГ и присылать туда апдейты
app.run_polling()

Keksik: /start
BOT: До Вас расстояние по Левенштейну = 4
Keksik: Привет
BOT: Вот так встреча
Keksik: Какдела
BOT: Поехали
Keksik: Хочу на работу
BOT: Может Machine Learning, чтобы я стал умнее?
Keksik: ок
BOT: Да - это совсем близко (вроде)
Keksik: Отбой
BOT: Увидимся!


RuntimeError: ignored